# This is the practice for logistic regression
# Goals:
## 1 - Fit logistic model (Sklearn and GLM- statsmodel)
## 2 - KPIs to evaluate binary classification model(logistic model)

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression #, LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
from sklearn.metrics import roc_curve, accuracy_score, roc_auc_score

import statsmodels.api as sm


from dmba import classificationSummary

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

Define paths to data sets. 

In [ ]:
DATA = Path('.').resolve().parents[1] / 'IntermediateLevel/DataScienceProgram/Class2/Practice/logit'
LOAN3000_CSV = DATA / 'loan3000.csv'
# LOAN_DATA_CSV = DATA / 'loan_data.csv.gz'

### Load Data and EDA

In [ ]:
loan_data = pd.read_csv(LOAN3000_CSV)
print(loan_data.shape)
print(loan_data.columns)
print(loan_data.head())

In [ ]:
loan_data = loan_data.drop(['Unnamed: 0'], axis=1).copy()
loan_data.columns=['outcome', 'purpose', 'dti', 'borrower_score', 'payment_inc_ratio']
loan_data['dv'] = [1 if out=="default" else 0 for out in loan_data['outcome']]

print(loan_data.shape)
print(loan_data.columns)
print(loan_data.head())

In [ ]:
for nm in loan_data.columns:
    print("\n{}".format(nm))
    print(loan_data[nm].value_counts())

In [ ]:
print("event rate is: {}".format(sum(loan_data['dv'])*100/len(loan_data)))

In [ ]:
loan_data.isnull().sum()

In [ ]:
numcols=list(loan_data.describe().columns)
num_stats = loan_data.describe().transpose() 
num_stats["nuniqueWna"]= loan_data[numcols].nunique(dropna=False) 
num_stats["nunique"]= loan_data[numcols].nunique()
num_stats

In [ ]:
loan_data.groupby(["purpose", 'dv']).size().reset_index(name='counts')

In [ ]:
purpose_risk = pd.concat([loan_data.groupby(["purpose"]).size(), loan_data.groupby(["purpose"]).sum()['dv']], axis=1)
purpose_risk.columns = ['count','default']
purpose_risk['default_rate']=purpose_risk['default']/purpose_risk['count']
print(purpose_risk.sort_values(['default_rate']))

In [ ]:
corr = loan_data[numcols].corr()
print("Correlation Matrix \n {}".format(corr))

ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True)

ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right')

#### This dataset do not have too much variables, therefore just did a light EDA. You can plot bar, histogram and bar/target, hist/targert. For this data just need to convert nominal to numerical

# Logistic regression

## Logistic Regression and the GLM
The package _scikit-learn_ has a specialised class for `LogisticRegression`. _Statsmodels_ has a more general method based on generalized linear model (GLM).

In scikit-learn, your target variable could be numerical or categorical.

In [ ]:
predictors = ['purpose', 'dti', 'borrower_score', 'payment_inc_ratio']
outcome = 'outcome'

loan, loan_test = train_test_split(loan_data, test_size=0.2)

X_train = pd.get_dummies(loan[predictors], prefix='', prefix_sep='', drop_first=True)
y_train = loan[outcome] 

X_test = pd.get_dummies(loan_test[predictors], prefix='', prefix_sep='', drop_first=True)
y_test = loan_test[outcome] 

#C=1e42,
logit_reg = LogisticRegression(penalty='l2', C=1e42, solver='liblinear')
logit_reg.fit(X_train, y_train)

print('intercept ', logit_reg.intercept_[0])
print('classes', logit_reg.classes_)
pd.DataFrame({'coeff': logit_reg.coef_[0]}, 
             index=X_train.columns)

In [ ]:
predictors = ['purpose', 'dti', 'borrower_score', 'payment_inc_ratio']
outcome = 'dv'

loan, loan_test = train_test_split(loan_data, test_size=0.2)

X_train = pd.get_dummies(loan[predictors], prefix='', prefix_sep='', drop_first=True)
y_train = loan[outcome] 

X_test = pd.get_dummies(loan_test[predictors], prefix='', prefix_sep='', drop_first=True)
y_test = loan_test[outcome] 

#C=1e42,
logit_reg2 = LogisticRegression(penalty='l2', C=1e42, solver='liblinear')
logit_reg2.fit(X_train, y_train)

print('intercept ', logit_reg2.intercept_[0])
print('classes', logit_reg2.classes_)
pd.DataFrame({'coeff': logit_reg2.coef_[0]}, 
             index=X_train.columns)

#### Predicted Values from Logistic Regression

In [ ]:
pred_train = pd.DataFrame(logit_reg.predict_proba(X_train),
                    columns=logit_reg.classes_)
pred_test = pd.DataFrame(logit_reg.predict_proba(X_test),
                    columns=logit_reg.classes_)
print(pred_train.describe())

print(pred_test.describe())

In [ ]:
pred_train = pd.DataFrame(logit_reg2.predict_proba(X_train),
                    columns=logit_reg2.classes_)
pred_test = pd.DataFrame(logit_reg2.predict_proba(X_test),
                    columns=logit_reg2.classes_)
print(pred_train.describe())

print(pred_test.describe())

## GLM Model - logistic regression
For comparison, here the GLM model using _statsmodels_. This method requires that the outcome is mapped to numbers. 
##### use GLM (general linear model) with the binomial family to fit a logistic regression
##### Notice: use this GLM module you can fit a series of model by changing the link function which specified in the family option

In [ ]:
logit_reg_sm = sm.GLM(y_train, sm.add_constant(X_train), 
                      family=sm.families.Binomial())
logit_result = logit_reg_sm.fit()
print(logit_result.summary())

# Evaluating Classification Models
## Confusion Matrix

In [ ]:
logit_reg2.predict(X_test)

In [ ]:
logit_reg2.predict_proba(X_test)

In [ ]:
# Confusion matrix
pred_y = logit_reg2.predict(X_test)
true_y = y_test 
true_pred = pd.DataFrame({'True': true_y, 'Pred': pred_y})
true_pred['Cnt'] = [1] *len(true_y)
confM=true_pred.groupby(['True', 'Pred']).size()

print("\nConfusion Matrix in 1D array\n {}".format(confM))

print("\nConfusion Matrix in 2 by 2 Table Format\n {}".format(pd.pivot_table(true_pred, values='Cnt', index=['True'],
                    columns=['Pred'], aggfunc=np.sum)))



conf_matorg = pd.DataFrame([[confM[1,1], confM[1,0]], [confM[0,1], confM[0,0]]],
                       index=['Y = default', 'Y = paid off'],
                       columns=['Yhat = default', 'Yhat = paid off'])
print("\nConfusion Matrix in 2 by 2 Re-orgnized Table Format\n {}".format(conf_matorg))

In [ ]:
print(confusion_matrix(y_test, logit_reg2.predict(X_test)))

The package _dmba_ contains the function `classificationSummary` that prints confusion matrix and accuracy for a classification model. 

In [ ]:
classificationSummary(y_test, logit_reg2.predict(X_test), 
                      class_names=logit_reg2.classes_)

## Precision, Recall, and Specificity
#### Precision = tp/(tp+fp)
#### Recall (Sensitivity) = tp/(tp+fn)
#### Specificity = tn/(tn+fp)

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, logit_reg2.predict(X_test)).ravel()
print('Precision:   {}'.format(tp/(tp+fp))) 
print('Recball:     {}'.format(tp/(tp+fn)))
print('Specificity: {}'.format(tn/(tn+fp)))

In [ ]:
conf_mat = confusion_matrix(y_test, logit_reg2.predict(X_test))
print('Precision', conf_mat[1, 1] / sum(conf_mat[:, 1]))
print('Recball', conf_mat[1, 1] / sum(conf_mat[1, :]))
print('Specificity', conf_mat[0, 0] / sum(conf_mat[0, :]))

The _scikit-learn_ function `precision_recall_fscore_support` returns
precision, recall, fbeta_score and support. 

In [ ]:
precision_recall_fscore_support(y_test, logit_reg2.predict(X_test), 
                                labels=[0, 1])

In [ ]:
from sklearn.metrics import classification_report
target_names = ['paid off', 'default']
print(classification_report(true_y, pred_y, target_names=target_names))

## ROC Curve
The function `roc_curve` in _Scikit-learn_ calculates all the information that is required for plotting a ROC curve.

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, (logit_reg2.predict_proba(X_test)[:, 1]), 
                                 pos_label=1)
roc_df = pd.DataFrame({'recall': tpr, 'specificity': 1 - fpr})

ax = roc_df.plot(x='specificity', y='recall', figsize=(4, 4), legend=False)
ax.set_ylim(0, 1)
ax.set_xlim(1, 0)
ax.plot((1, 0), (0, 1))
ax.set_xlabel('specificity')
ax.set_ylabel('recall')

plt.tight_layout()
plt.show()

## AUC
Accuracy can easily be calculated using the _scikit-learn_ function `accuracy_score`.

In [ ]:
print(np.sum(roc_df.recall[:-1] * np.diff(1 - roc_df.specificity)))
print(roc_auc_score(y_test, (logit_reg2.predict_proba(X_test)[:, 1])))

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, (logit_reg.predict_proba(X_test)[:,0]), 
                                 pos_label=1)
roc_df = pd.DataFrame({'recall': tpr, 'specificity': 1 - fpr})

ax = roc_df.plot(x='specificity', y='recall', figsize=(4, 4), legend=False)
ax.set_ylim(0, 1)
ax.set_xlim(1, 0)
# ax.plot((1, 0), (0, 1))
ax.set_xlabel('specificity')
ax.set_ylabel('recall')
ax.fill_between(roc_df.specificity, 0, roc_df.recall, alpha=0.3)


plt.tight_layout()
plt.show()

## F1 Score 
### F1 = 2 * Precision* Recall/(Precision + Recall)

In [ ]:
print('F1 SCORE for Default:  {}'.format(2*(tp/(tp+fp))*(tp/(tp+fn))/((tp/(tp+fp))+(tp/(tp+fn)))))